In [ ]:
THR_DIR = "/proj/verifiedmlsys-PG0/rohitd99/c3_throughput"

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
directory = "ksc10_cid6"
directory = os.path.join(THR_DIR, directory)

In [ ]:
all_data = {}
MIN_TS = None
MIN_TS_IDX = -1
for file in os.listdir(directory):
    df = pd.read_csv(os.path.join(directory, file), names=['ksc', 'epoch', 'ts'])
    df = df[['epoch', 'ts']]
    if MIN_TS is None:
        MIN_TS = df['ts'].min()
        MIN_TS_IDX = file
        all_data['epoch'] = df['epoch']
    else:
        if df['ts'].min() < MIN_TS:
            MIN_TS = df['ts'].min()
            MIN_TS_IDX = file
            all_data['epoch'] = df['epoch']
    
    actor_id = file.split('-')[-1].split('.')[0].replace('actor', '')
    all_data[f"actor{actor_id}"] = pd.Series(df['ts'])

In [ ]:
cleaned_df = pd.DataFrame(all_data)

actor_cols = list(cleaned_df)
actor_cols.remove('epoch')

cleaned_df[actor_cols] -= MIN_TS
last_actor_start_time = cleaned_df[actor_cols].loc[0].max()

In [ ]:
throughputs = []
n_epochs = []
for c in actor_cols:
    tmp = cleaned_df[cleaned_df[c] >= last_actor_start_time][['epoch', c]]
    if len(tmp) > 1:
        throughputs.append(
            (tmp['epoch'].iloc[-1] - tmp['epoch'].iloc[0]) / (tmp[c].iloc[-1] - tmp[c].iloc[0])
        )
        n_epochs.append((tmp['epoch'].iloc[-1] - tmp['epoch'].iloc[0]))
throughputs = np.asarray(throughputs)
throughputs = throughputs[~np.isnan(throughputs)] 
print(len(throughputs))

In [ ]:
round(np.mean(throughputs), 1)